In [1]:
import random
import math
import numpy as np
import numba as nb
from numba import njit

In [2]:
@njit
def funcion_p(a, h):
    valor1 = (2 * a + 1)
    valor2 = (a + 1) ** h
    valor3 = 0
    for k in range(0, a + 1):
        valor3 += k ** h
    valor3 *= 2
    return valor1 * valor2 - valor3

@njit
def prob_triangular_discreta(a, h, c, d):
    if d < c - a or d > c + a:
        return 0
    numerador = (a + 1) ** h - abs(d - c) ** h
    denominador = funcion_p(a, h)
    prob = numerador / denominador
    return prob

@njit
def va_triangular_discreta(a, h, c):
    u = np.random.random()
    # print(u)
    p_acum = 0
    for d in range(c - a, c + a + 1):
        prob = prob_triangular_discreta(a, h, c, d)
        if p_acum <= u < p_acum + prob:
            return d
        p_acum += prob
    return d

@njit
def esperanza_triangular_discreta(a, h, c):
    esperanza = 0
    for d in range(c - a, c + a + 1):
        esperanza += d * prob_triangular_discreta(a, h, c, d)
    return round(esperanza)

In [3]:
@njit
def a_value(t):
    if t <= 15:
        return 50
    elif t <= 30:
        return 86
    elif t <= 45:
        return 34
    elif t <= 60:
        return 22
    elif t <= 75:
        return 100
    return 66

@njit
def h_value(t):
    if t <= 15:
        return 5
    elif t <= 30:
        return 4
    elif t <= 45:
        return 4
    elif t <= 60:
        return 5
    elif t <= 75:
        return 3
    return 4

@njit
def c_value(t):
    if t <= 15:
        return 50
    elif t <= 30:
        return 86
    elif t <= 45:
        return 34
    elif t <= 60:
        return 22
    elif t <= 75:
        return 100
    return 66

@njit
def K_value(t):
    if t <= 30:
        return 180
    elif t <= 60:
        return 500
    return 250

@njit
def C_value(t):
    if (t % 7 == 0):
        return 18
    return 12

@njit
def H_value(t):
    return 1

@njit
def Q_value(t):
    return 18

In [4]:
@njit
def esperanza_triangular_discreta(a, h, c):
    esperanza = 0
    for d in range(c - a, c + a + 1):
        esperanza += d * prob_triangular_discreta(a, h, c, d)
    return round(esperanza)

In [5]:
@njit
def politica_optima(T, Q):
    array_size = (T+1, Q + 1)
    matriz_costos = np.zeros(array_size)
    matriz_decision = np.zeros(array_size)
    matriz_costos_inmediatos = np.zeros((Q + 1, Q + 1))

    for s in range(0, Q + 1):
        matriz_costos[T][s] = 0

    for t in range(T - 1, -1, -1):
        for s in range(0, Q + 1):
            #escoge acción óptima.
            minimum = np.inf
            x_value = -1
            for x in range(0, Q - s + 1):
                #costo inmediato
                CostoAccion = 0
                if x > 0:
                    CostoAccion += K_value(t+1) + C_value(t+1) * x
                for d in range(c_value(t+1) - a_value(t+1), c_value(t+1) + a_value(t+1) + 1):
                    CostoAccion += Q_value(t+1) * max(0, d - s - x) * prob_triangular_discreta(a_value(t+1), h_value(t+1), c_value(t+1), d)
                    CostoAccion += H_value(t+1) * max(0, s + x - d) * prob_triangular_discreta(a_value(t+1), h_value(t+1), c_value(t+1), d)
                #valor esperado futuro
                for d in range(c_value(t+1) - a_value(t+1), c_value(t+1) + a_value(t+1) + 1):
                    sfuturo = max(s + x - d, 0)
                    CostoAccion += matriz_costos[t+1][sfuturo] * prob_triangular_discreta(a_value(t+1), h_value(t+1), c_value(t+1), d)

                #Actualiza
                if CostoAccion< minimum:
                    minimum=CostoAccion
                    x_value=x

            matriz_costos[t][s] = minimum
            matriz_decision[t][s] = x_value


    return matriz_costos, matriz_decision

In [13]:
type(4)

int

In [14]:
T = 90
Q = 120
Q10 = int(Q*1.1)
Q20 = int(Q*1.2)
Q30 = int(Q*1.3)

In [ ]:
matriz_costos, matriz_decision = politica_optima(T, Q)

In [15]:

matriz_costos10, matriz_decision10 = politica_optima(T, Q10)

In [16]:
matriz_costos20, matriz_decision20 = politica_optima(T, Q20)    

In [17]:
matriz_costos30, matriz_decision30 = politica_optima(T, Q30)

In [18]:
def simulacion_aplicando_politica_optima(demanda, matriz_decision):
    stock_inicial = 40
    stock = stock_inicial
    costo_total = 0
    for t in range(1, T + 1):
        reposicion = matriz_decision[t - 1][stock]
        if reposicion > 0:
            costo_total += K_value(t) + C_value(t) * reposicion
        
        costo_total += Q_value(t) * max(0, demanda[t - 1] - stock - reposicion)
        costo_total += H_value(t) * max(0, stock + reposicion - demanda[t - 1])
        stock = int(max(stock + reposicion - demanda[t - 1], 0))

    return costo_total

In [19]:
M = 500

with open("demanda.csv", "r") as f:
    demanda = f.readlines()

CM = 0
CM10 = 0
CM20 = 0
CM30 = 0

for i in range(M):
    dem = demanda[i].split(",")
    for j in range(len(dem)):
        dem[j] = int(dem[j])
    CM += simulacion_aplicando_politica_optima(dem, matriz_decision)
    CM10 += simulacion_aplicando_politica_optima(dem, matriz_decision10)
    CM20 += simulacion_aplicando_politica_optima(dem, matriz_decision20)
    CM30 += simulacion_aplicando_politica_optima(dem, matriz_decision30)

print(f"Q = {Q}: {CM/M}")
print(f"Q = {Q10}: {CM10/M}")
print(f"Q = {Q20}: {CM20/M}")
print(f"Q = {Q30}: {CM30/M}")

Q = 120: 83021.854
Q = 132: 82150.04
Q = 144: 81500.736
Q = 156: 81006.28
